# 오늘은 LeNet 구조를 만들어봅시다


LeNet 구조는 CNN이며, 초기에 만들어진 모델입니다. 

2가지 모델(Sigmoid, ReLU)를 만들어 두 모델의 성능을 비교해봅시다.


## 1.우선 필요 라이브러리를 import 합니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets
import torchvision.transforms as transforms

import torch.optim as optim

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## 2. 딥러닝 모델을 설계할 때 활용하는 장비 확인

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 2.0.0  Device: cpu


## 3. MNIST 데이터 다운로드 

 1. Training data와 Test data 분리하기
 
 2. Training data를 Training data 와 Validation data로 분리하기

In [5]:
BATCH_SIZE = 64

transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.1307,), (0.3081,))
    ])

train_data = datasets.MNIST('data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('data', train=False, download=True, transform=transform)

train, val = torch.utils.data.random_split(train_data, [50000, 10000])

train_loader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)


100%|███████████████████████████| 9912422/9912422 [00:00<00:00, 14492647.92it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|███████████████████████████████| 28881/28881 [00:00<00:00, 24935301.32it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw




100%|███████████████████████████| 1648877/1648877 [00:00<00:00, 11771469.56it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████████████████████████████| 4542/4542 [00:00<00:00, 3253164.07it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



## 4. torch.nn을 이용하여 모델-1 만들기

   1) 아래의 그림 중 LeNet 구조를 구현 할 것
   
   2) Sigmoid 활성화 함수를 이용할 것
   
   
![](Comparison_image_neural_networks.svg.png)

In [3]:
import torch.nn as nn

class Model_1(nn.Module):
    def __init__(self):
        super(Model_1, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        return x

## 5. torch.nn을 이용하여 모델-2 만들기

   LeNet 모델에서 ReLU 활성화 함수를 사용하시요

In [4]:
class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


## 7. 학습 준비하기

1) 1 epoch를 학습할 수 있는 함수 만들기

2) Test와 Validation data의 정확도 계산할 수 있는 함수 만들기

In [11]:
def training_epoch(train_loader, network, loss_func, optimizer, epoch):
    train_losses = []
    train_correct = 0
    log_interval = 300
    
    for batch_idx, (image, label) in enumerate(train_loader):
        image, label = image.to(device), label.to(device)

        # 미분값의 초기화
        optimizer.zero_grad()

        # Forward propagration 계산하기.
        outputs = network(image)
        
        
        # Cross_entropy 함수를 적용하여 loss를 구하고 저장하기
        loss = loss_func(outputs, label)
        train_losses.append(loss.item())

        # training accuracy 정확도 구하기 위해 맞는 샘플 개수 세기
        pred = outputs.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(label.view_as(pred)).sum().item()

        # Gradinet 구하기
        loss.backward()

        # weight값 update 하기
        optimizer.step()

        # 학습 상황 출력
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.2f}%)]\tLoss: {:.6f}'
                  .format(epoch, batch_idx * len(label), len(train_loader.dataset),100. * batch_idx / len(train_loader),
                          loss.item()))
            
    return train_losses, train_correct

In [7]:
def test_epoch(test_loader, network, loss_func, val = False):
    correct = 0
    
    test_losses = []
    
    with torch.no_grad():
        for batch_idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)

            # Forward propagration 계산하기.
            outputs = network(image)

            # Cross_entropy 함수를 적용하여 loss를 구하기
            loss = loss_func(outputs, label)
            test_losses.append(loss.item())

            # Batch 별로 정확도 구하기
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()

        # 전체 정확도 구하기
        test_accuracy = 100. * correct / len(test_loader.dataset)

        #중간결과 출력
        if val is True:
                print('Validation set: Accuracy: {}/{} ({:.2f}%)\n'
              .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
        else:
            print('Test set: Accuracy: {}/{} ({:.2f}%)\n'
                  .format(correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
        
    return test_losses, test_accuracy


## 8. 위 정의된 함수로 학습 함수 만들기

Adam Optimizer를 사용하여 학습시키기

In [8]:
def training(network, learning_rate = 0.001):
    
    epoches = 15
    
    cls_loss = nn.CrossEntropyLoss()  # CrossEntropyLoss 함수 사용
    optimizer = optim.Adam(network.parameters(), lr=learning_rate)  # Adam optimizer 사용
    
    train_losses_per_epoch = []
    test_losses_per_epoch = []
    
    train_accuracies = []
    test_accuracies = []
    
    
    for epoch in range(epoches):
                
        # 모델를 학습 중이라고 선언하기
        network.train()
        
        train_losses, train_correct = training_epoch(train_loader,network,cls_loss,optimizer, epoch)
        
        # epoch 별로 loss 평균값, 정확도 구하기
        average_loss = np.mean(train_losses)
        train_losses_per_epoch.append(average_loss)
        
        train_accuracy = train_correct / len(train_loader.dataset) * 100
        train_accuracies.append(train_accuracy)
        
        # epoch 별로 정확도 출력
        print('\nTraining set: Accuracy: {}/{} ({:.2f}%)'
              .format(train_correct, len(train_loader.dataset),100. * train_correct / len(train_loader.dataset)))

        
        ### 학습 중에 test 결과 보기
        
        # 모델 test 중인 것을 선언하기
        network.eval()
        
        correct = 0
        with torch.no_grad():
            test_losses, test_accuracy = test_epoch(val_loader, network, cls_loss, True)

        test_losses_per_epoch.append(np.mean(test_losses))
        test_accuracies.append(test_accuracy)
        
    with torch.no_grad():
        test_losses, test_accuracy = test_epoch(test_loader, network, cls_loss, False)
        
    return train_losses_per_epoch, test_losses_per_epoch, train_accuracies, test_accuracies


In [20]:
network = Model_1().to(device)
print("2019125022 류재혁")
rlt_const = training(network)

2019125022 류재혁
Train Epoch: 0 [0/50000 (0.00%)]	Loss: 2.330729
Train Epoch: 0 [19200/50000 (38.36%)]	Loss: 0.203291
Train Epoch: 0 [38400/50000 (76.73%)]	Loss: 0.259508

Training set: Accuracy: 44721/50000 (89.44%)
Validation set: Accuracy: 9297/10000 (92.97%)

Train Epoch: 1 [0/50000 (0.00%)]	Loss: 0.199514
Train Epoch: 1 [19200/50000 (38.36%)]	Loss: 0.208589
Train Epoch: 1 [38400/50000 (76.73%)]	Loss: 0.137953

Training set: Accuracy: 47241/50000 (94.48%)
Validation set: Accuracy: 9463/10000 (94.63%)

Train Epoch: 2 [0/50000 (0.00%)]	Loss: 0.160542
Train Epoch: 2 [19200/50000 (38.36%)]	Loss: 0.207085
Train Epoch: 2 [38400/50000 (76.73%)]	Loss: 0.084850

Training set: Accuracy: 48004/50000 (96.01%)
Validation set: Accuracy: 9546/10000 (95.46%)

Train Epoch: 3 [0/50000 (0.00%)]	Loss: 0.097750
Train Epoch: 3 [19200/50000 (38.36%)]	Loss: 0.044264
Train Epoch: 3 [38400/50000 (76.73%)]	Loss: 0.160948

Training set: Accuracy: 48433/50000 (96.87%)
Validation set: Accuracy: 9624/10000 (96.24%

In [21]:
network = Model_2().to(device)
print("2019125022 류재혁")
rlt_const = training(network)

2019125022 류재혁
Train Epoch: 0 [0/50000 (0.00%)]	Loss: 2.335661
Train Epoch: 0 [19200/50000 (38.36%)]	Loss: 0.095989
Train Epoch: 0 [38400/50000 (76.73%)]	Loss: 0.136073

Training set: Accuracy: 45859/50000 (91.72%)
Validation set: Accuracy: 9457/10000 (94.57%)

Train Epoch: 1 [0/50000 (0.00%)]	Loss: 0.171159
Train Epoch: 1 [19200/50000 (38.36%)]	Loss: 0.038019
Train Epoch: 1 [38400/50000 (76.73%)]	Loss: 0.055777

Training set: Accuracy: 48177/50000 (96.35%)
Validation set: Accuracy: 9650/10000 (96.50%)

Train Epoch: 2 [0/50000 (0.00%)]	Loss: 0.238913
Train Epoch: 2 [19200/50000 (38.36%)]	Loss: 0.082533
Train Epoch: 2 [38400/50000 (76.73%)]	Loss: 0.136302

Training set: Accuracy: 48734/50000 (97.47%)
Validation set: Accuracy: 9661/10000 (96.61%)

Train Epoch: 3 [0/50000 (0.00%)]	Loss: 0.026779
Train Epoch: 3 [19200/50000 (38.36%)]	Loss: 0.103897
Train Epoch: 3 [38400/50000 (76.73%)]	Loss: 0.047951

Training set: Accuracy: 49028/50000 (98.06%)
Validation set: Accuracy: 9676/10000 (96.76%

## 9. 두모델의 성능을 비교하시오

정답)<br>
Sigmoid 함수의 정확도는 Accuracy: 9753/10000 (97.53%)이 나왔고<br>
ReLU 함수의 정확도는 Accuracy: 9757/10000 (97.57%)이 나왔다.<br>
따라서 ReLU함수가 성능이 더 좋다고 볼 수 있다.

In [19]:
print("2019125022 류재혁")

2019125022 류재혁
